In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Meta-learning pipeline with HVAC-aware features and flowering-period focus.
- Expects CSV columns: temp, humid, light, ac, heater, dehum, hum, label
- Sliding windows -> contrastive learning (unlabeled) + FOMAML with LLL + EWC (labeled)
- Encoder: LSTM on continuous features only (temp/humid/light)
- Additional HVAC features: mean on/off rate + toggle rate (abs(diff)) over time
- Gradient boost on flowering period with abnormal HVAC toggling
- TFLite export restricted to TFLITE_BUILTINS
- Includes Fisher matrix computation and loading for EWC
"""

import os, glob
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, models
import random

# =============================
# Hyperparameters
# =============================
DATA_GLOB = "../../../../lll_data/*.csv"
SEQ_LEN = 10
FEATURE_DIM = 64
BATCH_SIZE = 32
EPOCHS_CONTRASTIVE = 10
EPOCHS_META = 20
INNER_LR = 1e-2
META_LR = 1e-3
NUM_CLASSES = 3
NUM_TASKS = 5
SUPPORT_SIZE = 10
QUERY_SIZE = 20
REPLAY_CAPACITY = 1000
REPLAY_WEIGHT = 0.3
LAMBDA_EWC = 1e-3
FLOWERING_WEIGHT = 2.0  # gradient boost upper bound for flowering-focus

np.random.seed(42)
tf.random.set_seed(42)
random.seed(42)

# =============================
# 1) Load CSV -> Sliding windows
# =============================
X_labeled_list, y_labeled_list = [], []
X_unlabeled_list = []

for file in sorted(glob.glob(DATA_GLOB)):
    df = pd.read_csv(file).fillna(-1)
    data = df.values.astype(np.float32)
    feats, labels = data[:, :-1], data[:, -1]
    for i in range(len(data) - SEQ_LEN + 1):
        w_x = feats[i:i + SEQ_LEN]
        w_y = labels[i + SEQ_LEN - 1]
        if w_y == -1:
            X_unlabeled_list.append(w_x)
        else:
            X_labeled_list.append(w_x)
            y_labeled_list.append(int(w_y))

X_unlabeled = np.array(X_unlabeled_list, dtype=np.float32) if len(X_unlabeled_list) > 0 else np.empty((0,), dtype=np.float32)
if len(X_labeled_list) > 0:
    X_labeled = np.array(X_labeled_list, dtype=np.float32)
    y_labeled = np.array(y_labeled_list, dtype=np.int32)
else:
    X_labeled = np.empty((0, SEQ_LEN, X_unlabeled.shape[2] if X_unlabeled.size > 0 else 7), dtype=np.float32)
    y_labeled = np.empty((0,), dtype=np.int32)

NUM_FEATS = X_labeled.shape[2] if X_labeled.size > 0 else (X_unlabeled.shape[2] if X_unlabeled.size > 0 else 7)

if NUM_FEATS < 7:
    raise ValueError("Expected at least 7 features per timestep: [temp, humid, light, ac, heater, dehum, hum]. Found: %d" % NUM_FEATS)

# Index conventions
CONT_IDX = [0, 1, 2]   # temp, humid, light
HVAC_IDX = [3, 4, 5, 6]  # ac, heater, dehum, hum

# =============================
# 2) Contrastive learning
# =============================
def augment_window(x):
    """Only perturb continuous channels to keep binary HVAC channels intact."""
    x_aug = x.copy()
    x_aug[:, CONT_IDX] = x[:, CONT_IDX] + np.random.normal(0, 0.01, x[:, CONT_IDX].shape).astype(np.float32)
    return x_aug

def make_contrastive_pairs(X):
    anchors, positives = [], []
    for w in X:
        anchors.append(w)
        positives.append(augment_window(w))
    return np.stack(anchors).astype(np.float32), np.stack(positives).astype(np.float32)

class NTXentLoss(tf.keras.losses.Loss):
    def __init__(self, temperature=0.2):
        super().__init__()
        self.temperature = temperature
    def call(self, z_i, z_j):
        z_i = tf.math.l2_normalize(z_i, axis=1)
        z_j = tf.math.l2_normalize(z_j, axis=1)
        logits = tf.matmul(z_i, z_j, transpose_b=True) / self.temperature
        labels = tf.range(tf.shape(z_i)[0])
        loss_i = tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)
        loss_j = tf.keras.losses.sparse_categorical_crossentropy(labels, tf.transpose(logits), from_logits=True)
        return tf.reduce_mean(loss_i + loss_j)

# =============================
# 3) LSTM Encoder (unroll=True, continuous only)
# =============================
def build_lstm_encoder(seq_len, num_feats, feature_dim=FEATURE_DIM):
    inp = layers.Input(shape=(seq_len, num_feats))
    x_cont = layers.Lambda(lambda z: z[:, :, :3])(inp)  # [B,T,3]
    x = layers.LSTM(feature_dim, unroll=True)(x_cont)
    out = layers.Dense(feature_dim, activation="relu")(x)
    return models.Model(inp, out, name="lstm_encoder")

# =============================
# 4) Meta Model (Encoder + HVAC features)
# =============================
def build_meta_model(encoder, num_classes=NUM_CLASSES):
    inp = layers.Input(shape=(SEQ_LEN, NUM_FEATS))
    z_enc = encoder(inp)  # [B, FEATURE_DIM]

    # HVAC slice
    hvac = layers.Lambda(lambda z: z[:, :, 3:7])(inp)   # [B,T,4]
    hvac_mean = layers.Lambda(lambda z: tf.reduce_mean(z, axis=1))(hvac)  # [B,4]

    # Toggle rate via abs(diff)
    hvac_shift = layers.Lambda(lambda z: z[:, 1:, :])(hvac)      # [B,T-1,4]
    hvac_prev  = layers.Lambda(lambda z: z[:, :-1, :])(hvac)     # [B,T-1,4]
    hvac_diff  = layers.Lambda(lambda t: tf.abs(t[0] - t[1]))([hvac_shift, hvac_prev])  # [B,T-1,4]
    hvac_toggle_rate = layers.Lambda(lambda z: tf.reduce_mean(z, axis=1))(hvac_diff)    # [B,4]

    hvac_feat = layers.Concatenate()([hvac_mean, hvac_toggle_rate])  # [B,8]
    hvac_feat = layers.Dense(16, activation="relu")(hvac_feat)

    x = layers.Concatenate()([z_enc, hvac_feat])
    x = layers.Dense(64, activation="relu")(x)
    x = layers.Dense(32, activation="relu")(x)
    out = layers.Dense(NUM_CLASSES, activation="softmax")(x)
    return models.Model(inp, out, name="meta_lstm_classifier")

# Build models
lstm_encoder = build_lstm_encoder(SEQ_LEN, NUM_FEATS, FEATURE_DIM)
contrastive_opt = tf.keras.optimizers.Adam()
ntxent = NTXentLoss(temperature=0.2)

# Provide unlabeled fallback if none
if X_unlabeled.size == 0:
    X_unlabeled = np.random.randn(200, SEQ_LEN, NUM_FEATS).astype(np.float32)

anchors, positives = make_contrastive_pairs(X_unlabeled)
contrast_ds = tf.data.Dataset.from_tensor_slices((anchors, positives)).shuffle(2048).batch(BATCH_SIZE)

# Train contrastive
contrastive_loss_history = []
for ep in range(EPOCHS_CONTRASTIVE):
    for a, p in contrast_ds:
        with tf.GradientTape() as tape:
            za = lstm_encoder(a, training=True)
            zp = lstm_encoder(p, training=True)
            loss = ntxent(za, zp)
        grads = tape.gradient(loss, lstm_encoder.trainable_variables)
        contrastive_opt.apply_gradients(zip(grads, lstm_encoder.trainable_variables))
    contrastive_loss_history.append(float(loss.numpy()))
    print(f"[Contrastive] Epoch {ep+1}/{EPOCHS_CONTRASTIVE}, loss={float(loss.numpy()):.4f}")

# Meta model
meta_model = build_meta_model(lstm_encoder, NUM_CLASSES)
meta_optimizer = tf.keras.optimizers.Adam(META_LR)

def sample_tasks(X, y, num_tasks=NUM_TASKS, support_size=SUPPORT_SIZE, query_size=QUERY_SIZE):
    tasks = []
    n = len(X)
    if n < support_size + query_size:
        raise ValueError(f"Not enough labeled samples to build tasks: need {support_size+query_size}, got {n}")
    for _ in range(num_tasks):
        idx = np.random.choice(n, support_size + query_size, replace=False)
        X_support, y_support = X[idx[:support_size]], y[idx[:support_size]]
        X_query, y_query = X[idx[support_size:]], y[idx[support_size:]]
        tasks.append((X_support, y_support, X_query, y_query))
    return tasks

def inner_update(model, X_support, y_support, lr_inner=INNER_LR):
    with tf.GradientTape() as tape:
        preds_support = model(X_support, training=True)
        loss_support = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_support, preds_support))
    grads_inner = tape.gradient(loss_support, model.trainable_variables)
    updated_vars = [w - lr_inner * g for w, g in zip(model.trainable_variables, grads_inner)]
    return updated_vars

class ReplayBuffer:
    def __init__(self, capacity=REPLAY_CAPACITY):
        self.buffer = []
        self.capacity = capacity
        self.n_seen = 0
    def add(self, X, y):
        for xi, yi in zip(X, y):
            self.n_seen += 1
            if len(self.buffer) < self.capacity:
                self.buffer.append((xi, yi))
            else:
                r = np.random.randint(0, self.n_seen)
                if r < self.capacity:
                    self.buffer[r] = (xi, yi)
    def __len__(self):
        return len(self.buffer)
    def sample(self, batch_size):
        batch_size = min(batch_size, len(self.buffer))
        idxs = np.random.choice(len(self.buffer), batch_size, replace=False)
        X_s, y_s = zip(*[self.buffer[i] for i in idxs])
        return np.array(X_s), np.array(y_s)

memory = ReplayBuffer(capacity=REPLAY_CAPACITY)

# ===== Fisher Matrix Computation for EWC =====
def compute_fisher_matrix(model, X, y, num_samples=100):
    fisher = [tf.zeros_like(w) for w in model.trainable_variables]
    
    # Sample subset of data
    idx = np.random.choice(len(X), min(num_samples, len(X)), replace=False)
    X_sample = X[idx]
    y_sample = y[idx]
    
    for x, true_label in zip(X_sample, y_sample):
        with tf.GradientTape() as tape:
            prob = model(np.expand_dims(x, axis=0))[0, true_label]
            log_prob = tf.math.log(prob)
        grads = tape.gradient(log_prob, model.trainable_variables)
        fisher = [f + tf.square(g) for f, g in zip(fisher, grads)]
    #print("fisher matrix:",fisher)
    return [f / num_samples for f in fisher]

# ===== Helpers for flowering focus =====
def is_flowering_seq(x_seq, light_idx=2, th_light=550.0):
    light_mean = float(np.mean(x_seq[:, light_idx]))
    return light_mean >= th_light

def hvac_toggle_score(x_seq, hvac_slice=slice(3,7), th_toggle=0.15):
    hv = x_seq[:, hvac_slice]  # [T,4]
    if hv.shape[0] < 2:
        return 0.0, False
    diff = np.abs(hv[1:] - hv[:-1])   # [T-1,4]
    rate = float(diff.mean())
    return rate, rate >= th_toggle

def outer_update_with_lll(meta_model, meta_optimizer, tasks,
                          lr_inner=INNER_LR, replay_weight=REPLAY_WEIGHT,
                          lambda_ewc=LAMBDA_EWC, prev_weights=None, fisher_matrix=None):
    meta_grads = [tf.zeros_like(v) for v in meta_model.trainable_variables]
    query_acc_list, query_loss_list = [], []

    for X_support, y_support, X_query, y_query in tasks:
        orig_vars = [tf.identity(v) for v in meta_model.trainable_variables]

        # inner update
        updated_vars = inner_update(meta_model, X_support, y_support)
        for var, upd in zip(meta_model.trainable_variables, updated_vars):
            var.assign(upd)

        with tf.GradientTape() as tape:
            preds_q = meta_model(X_query, training=True)
            loss_q = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_query, preds_q))
            loss_total = loss_q

            # replay
            if len(memory) >= 8:
                X_old, y_old = memory.sample(batch_size=32)
                preds_old = meta_model(X_old, training=True)
                replay_loss = tf.reduce_mean(tf.keras.losses.sparse_categorical_crossentropy(y_old, preds_old))
                loss_total = (1 - replay_weight) * loss_total + replay_weight * replay_loss
            
            # EWC (using Fisher matrix)
            if prev_weights is not None and fisher_matrix is not None:
                ewc_loss = 0.0
                for w, pw, f in zip(meta_model.trainable_variables, prev_weights, fisher_matrix):
                    ewc_loss += tf.reduce_sum(f * tf.square(w - pw))
                loss_total += lambda_ewc * ewc_loss
                #for i, f in enumerate(prev_weights):
                #    print(f"Fisher matrix for variable {i} has shape: {f.shape}")

        grads = tape.gradient(loss_total, meta_model.trainable_variables)

        # ===== Flowering + HVAC toggling gradient boost =====
        flowering_mask = []
        toggle_scores = []
        for i in range(len(X_query)):
            x_seq = X_query[i]  # [T,D]
            flw = is_flowering_seq(x_seq, light_idx=2, th_light=550.0)
            tscore, tabove = hvac_toggle_score(x_seq, hvac_slice=slice(3,7), th_toggle=0.15)
            flowering_mask.append(bool(flw and tabove))
            toggle_scores.append(tscore)

        if any(flowering_mask):
            ratio = sum(flowering_mask) / len(flowering_mask)
            mean_toggle = np.mean([t for m,t in zip(flowering_mask, toggle_scores) if m]) if any(flowering_mask) else 0.0
            toggle_boost = min(1.0 + float(mean_toggle)*2.0, FLOWERING_WEIGHT)
            boost = 1.0 + (FLOWERING_WEIGHT - 1.0) * ratio
            total_boost = float(min(boost * toggle_boost, FLOWERING_WEIGHT))
            grads = [g * total_boost for g in grads]

        meta_grads = [mg + g / len(tasks) for mg, g in zip(meta_grads, grads)]

        q_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(preds_q, axis=1), y_query), tf.float32))
        query_acc_list.append(float(q_acc.numpy()))
        query_loss_list.append(float(loss_q.numpy()))

        # restore original vars
        for var, orig in zip(meta_model.trainable_variables, orig_vars):
            var.assign(orig)

        # update memory
        memory.add(X_support, y_support)
        memory.add(X_query, y_query)

    meta_optimizer.apply_gradients(zip(meta_grads, meta_model.trainable_variables))
    return float(np.mean(query_loss_list)), float(np.mean(query_acc_list)), [tf.identity(v) for v in meta_model.trainable_variables]

# ======= Train meta-learning =======
meta_loss_history, meta_acc_history = [], []
prev_weights = None
fisher_matrix = None

if X_labeled.size > 0:
    # Compute Fisher matrix on initial model
    fisher_matrix = compute_fisher_matrix(meta_model, X_labeled, y_labeled)
    
    for ep in range(EPOCHS_META):
        tasks = sample_tasks(X_labeled, y_labeled)
        loss, acc, prev_weights = outer_update_with_lll(
            meta_model, meta_optimizer, tasks, 
            prev_weights=prev_weights, fisher_matrix=fisher_matrix
        )
        meta_loss_history.append(loss)
        meta_acc_history.append(acc)
        print(f"[Meta] Epoch {ep+1}/{EPOCHS_META}, loss={loss:.4f}, acc={acc:.4f}")
else:
    print("Skip meta-learning: no labeled data.")

# =============================
# 5) Save/Load Fisher Matrix and Model Weights
# =============================
def save_ewc_assets(model, fisher_matrix, save_dir="ewc_assets"):
    os.makedirs(save_dir, exist_ok=True)
    
    # Save model weights
    model.save_weights(os.path.join(save_dir, "model_weights.h5"))
    
    # Save Fisher matrix
    fisher_numpy = [f.numpy() for f in fisher_matrix]
    np.savez(os.path.join(save_dir, "fisher_matrix.npz"), *fisher_numpy)
    
    print(f"EWC assets saved to {save_dir}")

def load_ewc_assets(model, save_dir="ewc_assets"):
    # Load model weights
    model.load_weights(os.path.join(save_dir, "model_weights.h5"))
    
    # Load Fisher matrix
    fisher_data = np.load(os.path.join(save_dir, "fisher_matrix.npz"))
    fisher_matrix = [tf.constant(arr) for arr in fisher_data.values()]
    
    print(f"EWC assets loaded from {save_dir}")
    return fisher_matrix

# Save assets if we have them
if fisher_matrix is not None:
    save_ewc_assets(meta_model, fisher_matrix)

# Example of loading (commented out since we just saved)
# loaded_fisher = load_ewc_assets(meta_model)

# =============================
# 6) TFLite export (BUILTINS only)
# =============================
def save_tflite(model, out_path):
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]
    tflite_model = converter.convert()
    with open(out_path, "wb") as f:
        f.write(tflite_model)
    print("Saved TFLite:", out_path)

# Save models
save_tflite(lstm_encoder, "lstm_encoder_contrastive.tflite")
if X_labeled.size > 0:
    save_tflite(meta_model, "meta_lstm_classifier.tflite")

print("Done.")

2025-08-20 11:50:09.310206: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-20 11:50:09.341509: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 11:50:09.625751: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2025-08-20 11:50:09.627533: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-08-20 11:50:10.989661: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not fin

[Contrastive] Epoch 1/10, loss=0.9898
[Contrastive] Epoch 2/10, loss=0.5418
[Contrastive] Epoch 3/10, loss=0.5915
[Contrastive] Epoch 4/10, loss=0.2044
[Contrastive] Epoch 5/10, loss=0.2235
[Contrastive] Epoch 6/10, loss=0.7443
[Contrastive] Epoch 7/10, loss=0.1826
[Contrastive] Epoch 8/10, loss=0.2345
[Contrastive] Epoch 9/10, loss=0.1065
[Contrastive] Epoch 10/10, loss=0.8159
[Meta] Epoch 1/20, loss=1.1271, acc=0.1500
[Meta] Epoch 2/20, loss=1.0721, acc=0.5800
[Meta] Epoch 3/20, loss=1.0344, acc=0.9500
[Meta] Epoch 4/20, loss=0.9804, acc=0.9500
[Meta] Epoch 5/20, loss=0.9547, acc=0.9100
[Meta] Epoch 6/20, loss=0.9108, acc=0.9500
[Meta] Epoch 7/20, loss=0.8705, acc=0.9700
[Meta] Epoch 8/20, loss=0.8298, acc=0.9600
[Meta] Epoch 9/20, loss=0.8004, acc=0.9600
[Meta] Epoch 10/20, loss=0.7624, acc=0.9600
[Meta] Epoch 11/20, loss=0.7191, acc=0.9600
[Meta] Epoch 12/20, loss=0.6933, acc=0.9400
[Meta] Epoch 13/20, loss=0.6309, acc=0.9400
[Meta] Epoch 14/20, loss=0.5872, acc=0.9200
[Meta] Epoch

INFO:tensorflow:Assets written to: /tmp/tmp6zh0_imc/assets
2025-08-20 11:50:31.197100: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-08-20 11:50:31.197171: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-08-20 11:50:31.198087: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp6zh0_imc
2025-08-20 11:50:31.200297: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-08-20 11:50:31.200329: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp6zh0_imc
2025-08-20 11:50:31.207815: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:375] MLIR V1 optimization pass is not enabled
2025-08-20 11:50:31.209082: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-08-20 11:50:31.237172: I tensorflow/cc/saved_model/loader.cc:215] Running initializatio

Saved TFLite: lstm_encoder_contrastive.tflite
INFO:tensorflow:Assets written to: /tmp/tmpuvpc9l03/assets


INFO:tensorflow:Assets written to: /tmp/tmpuvpc9l03/assets


Saved TFLite: meta_lstm_classifier.tflite
Done.


2025-08-20 11:50:33.816061: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-08-20 11:50:33.816123: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-08-20 11:50:33.816338: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmpuvpc9l03
2025-08-20 11:50:33.819921: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-08-20 11:50:33.819953: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmpuvpc9l03
2025-08-20 11:50:33.829369: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-08-20 11:50:33.862749: I tensorflow/cc/saved_model/loader.cc:215] Running initialization op on SavedModel bundle at path: /tmp/tmpuvpc9l03
2025-08-20 11:50:33.881361: I tensorflow/cc/saved_model/loader.cc:314] SavedModel load for tags { serve }; Status: success: OK. Took 65022 m